In [1]:
import requests
import datetime
import time
import csv
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint
from imblearn.over_sampling import SMOTE
pd.set_option('display.max_columns', None)
plt.style.use('seaborn')
import xgboost as xgb
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier
import seaborn as sns

Using TensorFlow backend.


In [32]:
models_df = pd.read_csv('models_df.csv')
models_df.drop('Unnamed: 0', axis=1, inplace=True)
models_df

,fight,winner,loser,date,cumultime_w,cumulStr_Att_fighter_w,cumulStr_Att_oponent_w,cumulTD_Att_fighter_w,cumulTD_Att_oponent_w,KDPM_w,KDPM_ratio_w,SLPM_w,SLPM_ratio_w,SAPM_w,SAPM_ratio_w,Str_Acc_w,Str_Def_w,TD_Acc_w,TD_Def_w,SubPM_w,PassPM_w,RevPM_w,Head_SLPM_w,Body_SLPM_w,Leg_SLPM_w,Dist_SLPM_w,Clinch_SLPM_w,Ground_SLPM_w,wins_w,losses_w,cumultime_l,cumulStr_Att_fighter_l,cumulStr_Att_oponent_l,cumulTD_Att_fighter_l,cumulTD_Att_oponent_l,KDPM_l,KDPM_ratio_l,SLPM_l,SLPM_ratio_l,SAPM_l,SAPM_ratio_l,Str_Acc_l,Str_Def_l,TD_Acc_l,TD_Def_l,SubPM_l,PassPM_l,RevPM_l,Head_SLPM_l,Body_SLPM_l,Leg_SLPM_l,Dist_SLPM_l,Clinch_SLPM_l,Ground_SLPM_l,wins_l,losses_l,implied_prob_w,implied_prob_l,prev_w,prev_l,bday_w,height_w,weight_w,reach_w,stance_w,bday_l,height_l,weight_l,reach_l,stance_l,age_w,age_l,weightclass,location
0,Charles Oliveira vs Jared Gordon,Charles-Oliveira,Jared-Gordon,2019-11-16,77.933333,953.0,1027.0,67.0,32.0,0.008083,0.000000,1.759867,0.387188,1.603631,0.338389,0.011882,0.012622,0.156914,0.160938,0.136264,0.107107,0.006547,0.955929,0.478180,0.325759,1.053005,0.362854,0.344008,15,8,26.016667,621.0,508.0,28.0,6.0,0.000000,0.000000,2.410269,0.394072,1.828344,0.219395,0.004365,0.005997,0.059311,0.166667,0.000000,0.134605,0.010982,2.059010,0.189633,0.161627,1.328531,0.240259,0.841479,3,2,0.793651,0.212766,23,5,1989-10-17,70.0,155.0,74.0,Orthodox,1988-09-06,69.0,155.0,68.0,Orthodox,30.0,31.0,Lightweight,"Sao Paulo, Sao Paulo, Brazil"
1,Charles Oliveira vs Jared Gordon,Charles-Oliveira,Jared-Gordon,2019-11-16,77.933333,953.0,1027.0,67.0,32.0,0.008083,0.000000,1.759867,0.387188,1.603631,0.338389,0.011882,0.012622,0.156914,0.160938,0.136264,0.107107,0.006547,0.955929,0.478180,0.325759,1.053005,0.362854,0.344008,15,8,26.016667,621.0,508.0,28.0,6.0,0.000000,0.000000,2.410269,0.394072,1.828344,0.219395,0.004365,0.005997,0.059311,0.166667,0.000000,0.134605,0.010982,2.059010,0.189633,0.161627,1.328531,0.240259,0.841479,3,2,0.793651,0.212766,23,5,1989-10-17,70.0,155.0,74.0,Orthodox,1988-09-06,69.0,155.0,68.0,Orthodox,30.0,31.0,Lightweight,"Sao Paulo, Sao Paulo, Brazil"
2,Charles Oliveira vs Jared Gordon,Charles-Oliveira,Jared-Gordon,2019-11-16,77.933333,953.0,1027.0,67.0,32.0,0.008083,0.000000,1.759867,0.387188,1.603631,0.338389,0.011882,0.012622,0.156914,0.160938,0.136264,0.107107,0.006547,0.955929,0.478180,0.325759,1.053005,0.362854,0.344008,15,8,26.016667,621.0,508.0,28.0,6.0,0.000000,0.000000,2.410269,0.394072,1.828344,0.219395,0.004365,0.005997,0.059311,0.166667,0.000000,0.134605,0.010982,2.059010,0.189633,0.161627,1.328531,0.240259,0.841479,3,2,0.793651,0.212766,23,5,1989-10-17,70.0,155.0,74.0,Orthodox,1988-09-06,69.0,155.0,68.0,Orthodox,30.0,31.0,Lightweight,"Sao Paulo, Sao Paulo, Brazil"
3,Charles Oliveira vs Jared Gordon,Charles-Oliveira,Jared-Gordon,2019-11-16,77.933333,953.0,1027.0,67.0,32.0,0.008083,0.000000,1.759867,0.387188,1.603631,0.338389,0.011882,0.012622,0.156914,0.160938,0.136264,0.107107,0.006547,0.955929,0.478180,0.325759,1.053005,0.362854,0.344008,15,8,26.016667,621.0,508.0,28.0,6.0,0.000000,0.000000,2.410269,0.394072,1.828344,0.219395,0.004365,0.005997,0.059311,0.166667,0.000000,0.134605,0.010982,2.059010,0.189633,0.161627,1.328531,0.240259,0.841479,3,2,0.793651,0.212766,23,5,1989-10-17,70.0,155.0,74.0,Orthodox,1988-09-06,69.0,155.0,68.0,Orthodox,30.0,31.0,Lightweight,"Sao Paulo, Sao Paulo, Brazil"
4,Joaquim Silva vs Jared Gordon,Joaquim-Silva,Jared-Gordon,2018-12-15,21.566667,412.0,417.0,1.0,20.0,0.081826,0.000000,1.378567,0.284387,1.117504,0.162303,0.004330,0.005376,0.000000,0.101603,0.006624,0.019872,0.000000,1.120232,0.218591,0.039744,0.863455,0.052992,0.462120,3,1,14.366667,348.0,244.0,14.0,3.0,0.000000,0.000000,2.151075,0.530367,1.550823,0.290996,0.004325,0.008236,0.068027,0.333333,0.000000,0.104224,0.000000,1.894013,0.153942,0.103119,0.919535,0.160940,1.070600,2,1,0.487805,0.529101,4,3,1989-02-05,68.0,155.0,69.0,Southpaw,1988-09-06,69.0,155.0,68.0,Orthodox,29.0,30.0,Lightweight,"Milwaukee, Wisconsin, USA"


In [3]:
# country = []
# for i in list(models_df['location']):
#     country.append(i[-5:])

# models_df['country'] = country

In [33]:
newest = pd.get_dummies(data=models_df, columns=['stance_w', 'stance_l', 'weightclass'])

In [34]:
newest = newest.sample(frac=1)

In [35]:
newest

,fight,winner,loser,date,cumultime_w,cumulStr_Att_fighter_w,cumulStr_Att_oponent_w,cumulTD_Att_fighter_w,cumulTD_Att_oponent_w,KDPM_w,KDPM_ratio_w,SLPM_w,SLPM_ratio_w,SAPM_w,SAPM_ratio_w,Str_Acc_w,Str_Def_w,TD_Acc_w,TD_Def_w,SubPM_w,PassPM_w,RevPM_w,Head_SLPM_w,Body_SLPM_w,Leg_SLPM_w,Dist_SLPM_w,Clinch_SLPM_w,Ground_SLPM_w,wins_w,losses_w,cumultime_l,cumulStr_Att_fighter_l,cumulStr_Att_oponent_l,cumulTD_Att_fighter_l,cumulTD_Att_oponent_l,KDPM_l,KDPM_ratio_l,SLPM_l,SLPM_ratio_l,SAPM_l,SAPM_ratio_l,Str_Acc_l,Str_Def_l,TD_Acc_l,TD_Def_l,SubPM_l,PassPM_l,RevPM_l,Head_SLPM_l,Body_SLPM_l,Leg_SLPM_l,Dist_SLPM_l,Clinch_SLPM_l,Ground_SLPM_l,wins_l,losses_l,implied_prob_w,implied_prob_l,prev_w,prev_l,bday_w,height_w,weight_w,reach_w,bday_l,height_l,weight_l,reach_l,age_w,age_l,location,stance_w_Orthodox,stance_w_Southpaw,stance_w_Switch,stance_l_Orthodox,stance_l_Southpaw,stance_l_Switch,weightclass_Bantamweight,weightclass_Catch Weight,weightclass_Featherweight,weightclass_Flyweight,weightclass_Heavyweight,weightclass_Light Heavyweight,weightclass_Lightweight,weightclass_Middleweight,weightclass_Welterweight,weightclass_Women's Bantamweight,weightclass_Women's Featherweight,weightclass_Women's Flyweight,weightclass_Women's Strawweight
1218,Erick Silva vs Neil Magny,Neil-Magny,Erick-Silva,2015-08-23,56.050000,980.0,528.0,44.0,54.0,0.005049,0.000000,1.760172,0.633346,0.704148,0.101334,0.004794,0.012334,0.120545,0.104416,0.005299,0.103628,0.009983,1.266204,0.288154,0.205814,0.861161,0.427905,0.471106,8,3,28.550000,233.0,377.0,8.0,33.0,0.060912,0.000000,2.311681,0.414093,1.815724,0.420818,0.025329,0.012281,0.208333,0.180014,0.099449,0.078695,0.000000,1.690404,0.564027,0.057249,0.665340,0.268043,1.378298,6,4,0.381679,0.628931,11,10,1987-08-03,75.0,170.0,80.0,1984-06-21,72.0,170.0,74.0,28.0,31.0,"Saskatoon, Saskatchewan, Canada",1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
512,Marc Diakiese vs Teemu Packalen,Marc-Diakiese,Teemu-Packalen,2017-03-18,12.666667,150.0,105.0,14.0,9.0,0.000000,0.000000,0.770234,0.215977,0.518134,0.209978,0.004523,0.011609,0.104396,0.148148,0.000000,0.033835,0.000000,0.557939,0.103494,0.108801,0.518797,0.192393,0.059045,2,0,7.400000,72.0,86.0,11.0,1.0,0.337838,0.000000,2.702703,0.109994,0.830116,0.166023,0.015545,0.017373,0.024793,1.000000,0.357143,0.511583,0.000000,2.567568,0.115830,0.019305,1.216216,0.000000,1.486486,1,1,0.666667,0.343643,2,2,1993-03-16,70.0,155.0,73.0,1987-05-22,73.0,155.0,75.0,24.0,29.0,"London, England, United Kingdom",1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
397,Tecia Torres vs Weili Zhang,Weili-Zhang,Tecia-Torres,2019-03-02,10.683333,252.0,135.0,5.0,2.0,0.000000,0.000000,2.173216,0.680004,0.683362,0.060052,0.004654,0.007212,0.166667,1.000000,0.050826,0.064198,0.000000,1.329391,0.378469,0.465356,1.025648,0.129726,1.017842,2,0,57.883333,1143.0,1139.0,41.0,28.0,0.002468,0.000000,1.480200,0.266552,1.007694,0.121249,0.003925,0.005307,0.077236,0.148810,0.009173,0.033853,0.000000,0.884992,0.288431,0.306777,1.161782,0.230268,0.088150,6,3,0.555556,0.416667,2,9,1989-08-13,64.0,115.0,63.0,1989-08-16,61.0,115.0,60.0,29.0,29.0,"Las Vegas, Nevada, USA",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
780,Urijah Faber vs Ricky Simon,Urijah-Faber,Ricky-Simon,2019-07-13,98.066667,1607.0,1919.0,89.0,48.0,0.011927,0.000000,1.179982,0.352180,0.991396,0.173518,0.004591,0.005525,0.051524,0.091232,0.022279,0.031277,0.006395,0.881764,0.217457,0.080760,0.706166,0.149947,0.323869,10,6,21.000000,293.0,490.0,32.0,33.0,0.013605,0.000000,0.768707,0.109598,1.020408,0.206880,0.004082,0.004102,0.044866,0.052342,0.006803,0.054422,0.006803,0.489796,0.170068,0.108844,0.571429,0.081633,0.115646,3,0,0.238095,0.714286,16,3,1979-05-14,66.0,135.0,67.0,1992-08-31,66.0,135.0,69.0,40.0,26.0,"Sacramento, California, USA",1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
23,Anthony Pettis vs Jim Miller,Anthony-Pettis,Jim-Miller,2017-07-08,59.150000,867.0,945.0,15.0,65.0,0.019067,0.000000,1.241935,0.217224,1.172125,0.185897,0.005827,0.006997,0.044444,0.064790,0.016318,0

In [36]:
newest.columns

Index(['fight', 'winner', 'loser', 'date', 'cumultime_w',
       'cumulStr_Att_fighter_w', 'cumulStr_Att_oponent_w',
       'cumulTD_Att_fighter_w', 'cumulTD_Att_oponent_w', 'KDPM_w',
       'KDPM_ratio_w', 'SLPM_w', 'SLPM_ratio_w', 'SAPM_w', 'SAPM_ratio_w',
       'Str_Acc_w', 'Str_Def_w', 'TD_Acc_w', 'TD_Def_w', 'SubPM_w', 'PassPM_w',
       'RevPM_w', 'Head_SLPM_w', 'Body_SLPM_w', 'Leg_SLPM_w', 'Dist_SLPM_w',
       'Clinch_SLPM_w', 'Ground_SLPM_w', 'wins_w', 'losses_w', 'cumultime_l',
       'cumulStr_Att_fighter_l', 'cumulStr_Att_oponent_l',
       'cumulTD_Att_fighter_l', 'cumulTD_Att_oponent_l', 'KDPM_l',
       'KDPM_ratio_l', 'SLPM_l', 'SLPM_ratio_l', 'SAPM_l', 'SAPM_ratio_l',
       'Str_Acc_l', 'Str_Def_l', 'TD_Acc_l', 'TD_Def_l', 'SubPM_l', 'PassPM_l',
       'RevPM_l', 'Head_SLPM_l', 'Body_SLPM_l', 'Leg_SLPM_l', 'Dist_SLPM_l',
       'Clinch_SLPM_l', 'Ground_SLPM_l', 'wins_l', 'losses_l',
       'implied_prob_w', 'implied_prob_l', 'prev_w', 'prev_l', 'bday_w',
       'heig

In [37]:
winners = newest[['fight', 'date', 'winner', 'cumultime_w',
       'cumulStr_Att_fighter_w', 'cumulStr_Att_oponent_w',
       'cumulTD_Att_fighter_w', 'cumulTD_Att_oponent_w', 'KDPM_w',
       'KDPM_ratio_w', 'SLPM_w', 'SLPM_ratio_w', 'SAPM_w', 'SAPM_ratio_w',
       'Str_Acc_w', 'Str_Def_w', 'TD_Acc_w', 'TD_Def_w', 'SubPM_w', 'PassPM_w',
       'RevPM_w', 'Head_SLPM_w', 'Body_SLPM_w', 'Leg_SLPM_w', 'Dist_SLPM_w',
       'Clinch_SLPM_w', 'Ground_SLPM_w', 'wins_w', 'losses_w', 'implied_prob_w',
       'prev_w', 'height_w', 'weight_w', 'reach_w', 'age_w', 'stance_w_Orthodox',
       'stance_w_Southpaw', 'stance_w_Switch']]

In [38]:
losers = newest[['fight', 'date', 'loser', 'cumultime_l',
       'cumulStr_Att_fighter_l', 'cumulStr_Att_oponent_l',
       'cumulTD_Att_fighter_l', 'cumulTD_Att_oponent_l', 'KDPM_l',
       'KDPM_ratio_l', 'SLPM_l', 'SLPM_ratio_l', 'SAPM_l', 'SAPM_ratio_l',
       'Str_Acc_l', 'Str_Def_l', 'TD_Acc_l', 'TD_Def_l', 'SubPM_l', 'PassPM_l',
       'RevPM_l', 'Head_SLPM_l', 'Body_SLPM_l', 'Leg_SLPM_l', 'Dist_SLPM_l',
       'Clinch_SLPM_l', 'Ground_SLPM_l', 'wins_l', 'losses_l', 'implied_prob_l',
       'prev_l', 'height_l', 'weight_l', 'reach_l', 'age_l', 'stance_l_Orthodox',
       'stance_l_Southpaw', 'stance_l_Switch']]

In [39]:
len(winners)

1464

In [40]:
len(losers)

1464

In [41]:
winners.rename(columns={'winner': 'fighter'}, inplace=True)
losers.rename(columns={'loser': 'fighter'}, inplace=True)

In [42]:
winners

,fight,date,fighter,cumultime_w,cumulStr_Att_fighter_w,cumulStr_Att_oponent_w,cumulTD_Att_fighter_w,cumulTD_Att_oponent_w,KDPM_w,KDPM_ratio_w,SLPM_w,SLPM_ratio_w,SAPM_w,SAPM_ratio_w,Str_Acc_w,Str_Def_w,TD_Acc_w,TD_Def_w,SubPM_w,PassPM_w,RevPM_w,Head_SLPM_w,Body_SLPM_w,Leg_SLPM_w,Dist_SLPM_w,Clinch_SLPM_w,Ground_SLPM_w,wins_w,losses_w,implied_prob_w,prev_w,height_w,weight_w,reach_w,age_w,stance_w_Orthodox,stance_w_Southpaw,stance_w_Switch
1218,Erick Silva vs Neil Magny,2015-08-23,Neil-Magny,56.050000,980.0,528.0,44.0,54.0,0.005049,0.000000,1.760172,0.633346,0.704148,0.101334,0.004794,0.012334,0.120545,0.104416,0.005299,0.103628,0.009983,1.266204,0.288154,0.205814,0.861161,0.427905,0.471106,8,3,0.381679,11,75.0,170.0,80.0,28.0,1,0,0
512,Marc Diakiese vs Teemu Packalen,2017-03-18,Marc-Diakiese,12.666667,150.0,105.0,14.0,9.0,0.000000,0.000000,0.770234,0.215977,0.518134,0.209978,0.004523,0.011609,0.104396,0.148148,0.000000,0.033835,0.000000,0.557939,0.103494,0.108801,0.518797,0.192393,0.059045,2,0,0.666667,2,70.0,155.0,73.0,24.0,1,0,0
397,Tecia Torres vs Weili Zhang,2019-03-02,Weili-Zhang,10.683333,252.0,135.0,5.0,2.0,0.000000,0.000000,2.173216,0.680004,0.683362,0.060052,0.004654,0.007212,0.166667,1.000000,0.050826,0.064198,0.000000,1.329391,0.378469,0.465356,1.025648,0.129726,1.017842,2,0,0.555556,2,64.0,115.0,63.0,29.0,0,0,1
780,Urijah Faber vs Ricky Simon,2019-07-13,Urijah-Faber,98.066667,1607.0,1919.0,89.0,48.0,0.011927,0.000000,1.179982,0.352180,0.991396,0.173518,0.004591,0.005525,0.051524,0.091232,0.022279,0.031277,0.006395,0.881764,0.217457,0.080760,0.706166,0.149947,0.323869,10,6,0.238095,16,66.0,135.0,67.0,40.0,1,0,0
23,Anthony Pettis vs Jim Miller,2017-07-08,Anthony-Pettis,59.150000,867.0,945.0,15.0,65.0,0.019067,0.000000,1.241935,0.217224,1.172125,0.185897,0.005827,0.006997,0.044444,0.064790,0.016318,0.024383,0.028011,0.878335,0.227192,0.136408,1.058264,0.048451,0.135220,6,5,0.675676,11,70.0,170.0,72.0,30.0,1,0,0
386,Yair Rodriguez vs Alex Caceres,2016-08-06,Yair-Rodriguez,24.250000,340.0,261.0,18.0,20.0,0.012688,0.000000,1.153733,0.308279,0.675654,0.101955,0.005710,0.009397,0.141975,0.140476,0.041237,0.073411,0.011782,0.610853,0.283222,0.259658,0.827461,0.088365,0.237906,4,0,0.724638,4,71.0,145.0,71.0,23.0,1,0,0
668,Donald Cerrone vs Mike Perry,2018-11-10,Donald-Cerrone,134.150000,2471.0,2588.0,65.0,57.0,0.037986,0.022363,1.820667,0.244604,2.052632,0.310910,0.005634,0.005881,0.125775,0.245865,0.019975,0.041051,0.002613,0.895784,0.435111,0.489772,1.520768,0.145459,0.154440,20,8,0.363636,28,73.0,155.0,73.0,35.0,1,0,0
335,Michael Johnson vs Khabib Nurmagomedov,2016-11-12,Khabib-Nurmagomedov,38.683333,581.0,411.0,75.0,12.0,0.019742,0.000000,1.443705,0.832486,0.522134,0.090762,0.006003,0.012059,0.036431,0.319444,0.013640,0.144331,0.000000,1.247599,0.111088,0.085018,0.469735,0.090816,0.883154,7,0,0.735294,7,70.0,155.0,70.0,28.0,1,0,0
1082,Nate Marquardt vs Cezar Ferreira,2017-11-11,Cezar-Ferreira,54.550000,463.0,511.0,29.0,13.0,0.005238,0.000000,0.685717,0.190023,0.924717,0.290569,0.009524,0.012647,0.161494,0.365385,0.034399,0.044523,0.000000,0.446521,0.157264,0.081932,0.584622,0.019911,0.081184,7,4,0.729927,11,73.0,185.0,78.0,32.0,0,1,0
1313,Joe Soto vs Brett Johns,2017-12-01,Brett-Johns,14.000000,184.0,243.0,20.0,0.0,0.000000,0.000000,1.071429,0.208895,0.826531,0.112974,0.006255,0.005525,0.066667,NaN,0.010204,0.051020,0.000000,0.989796,0.051020,0.030612,0.469388,0.000000,0.602041,2,0,0.617284,2,67.0,135.0,71.0,25.0,1,0,0


In [43]:
losers

,fight,date,fighter,cumultime_l,cumulStr_Att_fighter_l,cumulStr_Att_oponent_l,cumulTD_Att_fighter_l,cumulTD_Att_oponent_l,KDPM_l,KDPM_ratio_l,SLPM_l,SLPM_ratio_l,SAPM_l,SAPM_ratio_l,Str_Acc_l,Str_Def_l,TD_Acc_l,TD_Def_l,SubPM_l,PassPM_l,RevPM_l,Head_SLPM_l,Body_SLPM_l,Leg_SLPM_l,Dist_SLPM_l,Clinch_SLPM_l,Ground_SLPM_l,wins_l,losses_l,implied_prob_l,prev_l,height_l,weight_l,reach_l,age_l,stance_l_Orthodox,stance_l_Southpaw,stance_l_Switch
1218,Erick Silva vs Neil Magny,2015-08-23,Erick-Silva,28.550000,233.0,377.0,8.0,33.0,0.060912,0.000000,2.311681,0.414093,1.815724,0.420818,0.025329,0.012281,0.208333,0.180014,0.099449,0.078695,0.000000,1.690404,0.564027,0.057249,0.665340,0.268043,1.378298,6,4,0.628931,10,72.0,170.0,74.0,31.0,1,0,0
512,Marc Diakiese vs Teemu Packalen,2017-03-18,Teemu-Packalen,7.400000,72.0,86.0,11.0,1.0,0.337838,0.000000,2.702703,0.109994,0.830116,0.166023,0.015545,0.017373,0.024793,1.000000,0.357143,0.511583,0.000000,2.567568,0.115830,0.019305,1.216216,0.000000,1.486486,1,1,0.343643,2,73.0,155.0,75.0,29.0,1,0,0
397,Tecia Torres vs Weili Zhang,2019-03-02,Tecia-Torres,57.883333,1143.0,1139.0,41.0,28.0,0.002468,0.000000,1.480200,0.266552,1.007694,0.121249,0.003925,0.005307,0.077236,0.148810,0.009173,0.033853,0.000000,0.884992,0.288431,0.306777,1.161782,0.230268,0.088150,6,3,0.416667,9,61.0,115.0,60.0,29.0,1,0,0
780,Urijah Faber vs Ricky Simon,2019-07-13,Ricky-Simon,21.000000,293.0,490.0,32.0,33.0,0.013605,0.000000,0.768707,0.109598,1.020408,0.206880,0.004082,0.004102,0.044866,0.052342,0.006803,0.054422,0.006803,0.489796,0.170068,0.108844,0.571429,0.081633,0.115646,3,0,0.714286,3,66.0,135.0,69.0,26.0,1,0,0
23,Anthony Pettis vs Jim Miller,2017-07-08,Jim-Miller,132.850000,1902.0,1869.0,74.0,61.0,0.008944,0.000000,1.018091,0.301817,0.897850,0.268107,0.004820,0.008376,0.137709,0.156740,0.059868,0.058093,0.005132,0.637816,0.192071,0.188204,0.674569,0.161144,0.182378,17,8,0.341297,25,68.0,155.0,71.0,33.0,0,1,0
386,Yair Rodriguez vs Alex Caceres,2016-08-06,Alex-Caceres,69.100000,1185.0,896.0,10.0,63.0,0.004244,0.000000,1.393131,0.384810,0.978804,0.163301,0.005183,0.008324,0.333333,0.089683,0.036436,0.037027,0.021068,0.863879,0.330884,0.198368,1.153009,0.153599,0.086523,7,6,0.289855,13,70.0,145.0,73.0,28.0,0,1,0
668,Donald Cerrone vs Mike Perry,2018-11-10,Mike-Perry,41.700000,783.0,943.0,8.0,18.0,0.064633,0.000000,1.696012,0.269137,1.611790,0.201156,0.004959,0.004952,0.239583,0.241667,0.000000,0.041212,0.000000,1.103540,0.220863,0.371609,1.248995,0.254775,0.192242,5,3,0.649351,8,70.0,170.0,71.0,27.0,1,0,0
335,Michael Johnson vs Khabib Nurmagomedov,2016-11-12,Michael-Johnson,80.466667,1854.0,1482.0,13.0,58.0,0.031919,0.012428,1.480057,0.308952,1.257626,0.253126,0.002845,0.006152,0.403846,0.181366,0.000000,0.000000,0.003551,0.829071,0.358896,0.292090,1.214812,0.144234,0.121012,9,6,0.271739,15,70.0,155.0,73.0,30.0,0,1,0
1082,Nate Marquardt vs Cezar Ferreira,2017-11-11,Nate-Marquardt,109.250000,1108.0,1043.0,52.0,52.0,0.053192,0.000000,1.181680,0.651817,0.774472,0.228610,0.011025,0.013551,0.191621,0.182158,0.025957,0.045873,0.006714,0.714060,0.232936,0.234684,0.679408,0.260250,0.242022,12,11,0.285714,23,72.0,185.0,74.0,38.0,1,0,0
1313,Joe Soto vs Brett Johns,2017-12-01,Joe-Soto,29.216667,568.0,1115.0,17.0,20.0,0.000000,0.000000,1.294496,0.134138,2.245891,0.802553,0.005086,0.003537,0.145658,0.087121,0.062625,0.051232,0.000000,1.035366,0.089716,0.169415,1.156636,0.014669,0.123191,3,3,0.374532,6,66.0,135.0,65.0,30.0,1,0,0


In [44]:
winners.columns = [c.replace('_w', '') for c in winners]
losers.columns = [c.replace('_l', '') for c in losers]

In [62]:
fighter1 = pd.DataFrame()
fighter1 = fighter1.append(winners[0:len(newest)//2])
fighter1 = fighter1.append(losers[len(newest)//2:])

In [63]:
fighter1

,fight,date,fighter,cumultime,cumulStr_Att_fighter,cumulStr_Att_oponent,cumulTD_Att_fighter,cumulTD_Att_oponent,KDPM,KDPM_ratio,SLPM,SLPM_ratio,SAPM,SAPM_ratio,Str_Acc,Str_Def,TD_Acc,TD_Def,SubPM,PassPM,RevPM,Head_SLPM,Body_SLPM,Leg_SLPM,Dist_SLPM,Clinch_SLPM,Ground_SLPM,wins,losses,implied_prob,prev,height,weight,reach,age,stance_Orthodox,stance_Southpaw,stance_Switch
1218,Erick Silva vs Neil Magny,2015-08-23,Neil-Magny,56.050000,980.0,528.0,44.0,54.0,0.005049,0.000000,1.760172,0.633346,0.704148,0.101334,0.004794,0.012334,0.120545,0.104416,0.005299,0.103628,0.009983,1.266204,0.288154,0.205814,0.861161,0.427905,0.471106,8,3,0.381679,11,75.0,170.0,80.0,28.0,1,0,0
512,Marc Diakiese vs Teemu Packalen,2017-03-18,Marc-Diakiese,12.666667,150.0,105.0,14.0,9.0,0.000000,0.000000,0.770234,0.215977,0.518134,0.209978,0.004523,0.011609,0.104396,0.148148,0.000000,0.033835,0.000000,0.557939,0.103494,0.108801,0.518797,0.192393,0.059045,2,0,0.666667,2,70.0,155.0,73.0,24.0,1,0,0
397,Tecia Torres vs Weili Zhang,2019-03-02,Weili-Zhang,10.683333,252.0,135.0,5.0,2.0,0.000000,0.000000,2.173216,0.680004,0.683362,0.060052,0.004654,0.007212,0.166667,1.000000,0.050826,0.064198,0.000000,1.329391,0.378469,0.465356,1.025648,0.129726,1.017842,2,0,0.555556,2,64.0,115.0,63.0,29.0,0,0,1
780,Urijah Faber vs Ricky Simon,2019-07-13,Urijah-Faber,98.066667,1607.0,1919.0,89.0,48.0,0.011927,0.000000,1.179982,0.352180,0.991396,0.173518,0.004591,0.005525,0.051524,0.091232,0.022279,0.031277,0.006395,0.881764,0.217457,0.080760,0.706166,0.149947,0.323869,10,6,0.238095,16,66.0,135.0,67.0,40.0,1,0,0
23,Anthony Pettis vs Jim Miller,2017-07-08,Anthony-Pettis,59.150000,867.0,945.0,15.0,65.0,0.019067,0.000000,1.241935,0.217224,1.172125,0.185897,0.005827,0.006997,0.044444,0.064790,0.016318,0.024383,0.028011,0.878335,0.227192,0.136408,1.058264,0.048451,0.135220,6,5,0.675676,11,70.0,170.0,72.0,30.0,1,0,0
386,Yair Rodriguez vs Alex Caceres,2016-08-06,Yair-Rodriguez,24.250000,340.0,261.0,18.0,20.0,0.012688,0.000000,1.153733,0.308279,0.675654,0.101955,0.005710,0.009397,0.141975,0.140476,0.041237,0.073411,0.011782,0.610853,0.283222,0.259658,0.827461,0.088365,0.237906,4,0,0.724638,4,71.0,145.0,71.0,23.0,1,0,0
668,Donald Cerrone vs Mike Perry,2018-11-10,Donald-Cerrone,134.150000,2471.0,2588.0,65.0,57.0,0.037986,0.022363,1.820667,0.244604,2.052632,0.310910,0.005634,0.005881,0.125775,0.245865,0.019975,0.041051,0.002613,0.895784,0.435111,0.489772,1.520768,0.145459,0.154440,20,8,0.363636,28,73.0,155.0,73.0,35.0,1,0,0
335,Michael Johnson vs Khabib Nurmagomedov,2016-11-12,Khabib-Nurmagomedov,38.683333,581.0,411.0,75.0,12.0,0.019742,0.000000,1.443705,0.832486,0.522134,0.090762,0.006003,0.012059,0.036431,0.319444,0.013640,0.144331,0.000000,1.247599,0.111088,0.085018,0.469735,0.090816,0.883154,7,0,0.735294,7,70.0,155.0,70.0,28.0,1,0,0
1082,Nate Marquardt vs Cezar Ferreira,2017-11-11,Cezar-Ferreira,54.550000,463.0,511.0,29.0,13.0,0.005238,0.000000,0.685717,0.190023,0.924717,0.290569,0.009524,0.012647,0.161494,0.365385,0.034399,0.044523,0.000000,0.446521,0.157264,0.081932,0.584622,0.019911,0.081184,7,4,0.729927,11,73.0,185.0,78.0,32.0,0,1,0
1313,Joe Soto vs Brett Johns,2017-12-01,Brett-Johns,14.000000,184.0,243.0,20.0,0.0,0.000000,0.000000,1.071429,0.208895,0.826531,0.112974,0.006255,0.005525,0.066667,NaN,0.010204,0.051020,0.000000,0.989796,0.051020,0.030612,0.469388,0.000000,0.602041,2,0,0.617284,2,67.0,135.0,71.0,25.0,1,0,0


In [64]:
fighter2 = pd.DataFrame()
fighter2 = fighter2.append(losers[0:len(newest)//2])
fighter2 = fighter2.append(winners[len(newest)//2:])

In [65]:
fighter2

,fight,date,fighter,cumultime,cumulStr_Att_fighter,cumulStr_Att_oponent,cumulTD_Att_fighter,cumulTD_Att_oponent,KDPM,KDPM_ratio,SLPM,SLPM_ratio,SAPM,SAPM_ratio,Str_Acc,Str_Def,TD_Acc,TD_Def,SubPM,PassPM,RevPM,Head_SLPM,Body_SLPM,Leg_SLPM,Dist_SLPM,Clinch_SLPM,Ground_SLPM,wins,losses,implied_prob,prev,height,weight,reach,age,stance_Orthodox,stance_Southpaw,stance_Switch
1218,Erick Silva vs Neil Magny,2015-08-23,Erick-Silva,28.550000,233.0,377.0,8.0,33.0,0.060912,0.000000,2.311681,0.414093,1.815724,0.420818,0.025329,0.012281,0.208333,0.180014,0.099449,0.078695,0.000000,1.690404,0.564027,0.057249,0.665340,0.268043,1.378298,6,4,0.628931,10,72.0,170.0,74.0,31.0,1,0,0
512,Marc Diakiese vs Teemu Packalen,2017-03-18,Teemu-Packalen,7.400000,72.0,86.0,11.0,1.0,0.337838,0.000000,2.702703,0.109994,0.830116,0.166023,0.015545,0.017373,0.024793,1.000000,0.357143,0.511583,0.000000,2.567568,0.115830,0.019305,1.216216,0.000000,1.486486,1,1,0.343643,2,73.0,155.0,75.0,29.0,1,0,0
397,Tecia Torres vs Weili Zhang,2019-03-02,Tecia-Torres,57.883333,1143.0,1139.0,41.0,28.0,0.002468,0.000000,1.480200,0.266552,1.007694,0.121249,0.003925,0.005307,0.077236,0.148810,0.009173,0.033853,0.000000,0.884992,0.288431,0.306777,1.161782,0.230268,0.088150,6,3,0.416667,9,61.0,115.0,60.0,29.0,1,0,0
780,Urijah Faber vs Ricky Simon,2019-07-13,Ricky-Simon,21.000000,293.0,490.0,32.0,33.0,0.013605,0.000000,0.768707,0.109598,1.020408,0.206880,0.004082,0.004102,0.044866,0.052342,0.006803,0.054422,0.006803,0.489796,0.170068,0.108844,0.571429,0.081633,0.115646,3,0,0.714286,3,66.0,135.0,69.0,26.0,1,0,0
23,Anthony Pettis vs Jim Miller,2017-07-08,Jim-Miller,132.850000,1902.0,1869.0,74.0,61.0,0.008944,0.000000,1.018091,0.301817,0.897850,0.268107,0.004820,0.008376,0.137709,0.156740,0.059868,0.058093,0.005132,0.637816,0.192071,0.188204,0.674569,0.161144,0.182378,17,8,0.341297,25,68.0,155.0,71.0,33.0,0,1,0
386,Yair Rodriguez vs Alex Caceres,2016-08-06,Alex-Caceres,69.100000,1185.0,896.0,10.0,63.0,0.004244,0.000000,1.393131,0.384810,0.978804,0.163301,0.005183,0.008324,0.333333,0.089683,0.036436,0.037027,0.021068,0.863879,0.330884,0.198368,1.153009,0.153599,0.086523,7,6,0.289855,13,70.0,145.0,73.0,28.0,0,1,0
668,Donald Cerrone vs Mike Perry,2018-11-10,Mike-Perry,41.700000,783.0,943.0,8.0,18.0,0.064633,0.000000,1.696012,0.269137,1.611790,0.201156,0.004959,0.004952,0.239583,0.241667,0.000000,0.041212,0.000000,1.103540,0.220863,0.371609,1.248995,0.254775,0.192242,5,3,0.649351,8,70.0,170.0,71.0,27.0,1,0,0
335,Michael Johnson vs Khabib Nurmagomedov,2016-11-12,Michael-Johnson,80.466667,1854.0,1482.0,13.0,58.0,0.031919,0.012428,1.480057,0.308952,1.257626,0.253126,0.002845,0.006152,0.403846,0.181366,0.000000,0.000000,0.003551,0.829071,0.358896,0.292090,1.214812,0.144234,0.121012,9,6,0.271739,15,70.0,155.0,73.0,30.0,0,1,0
1082,Nate Marquardt vs Cezar Ferreira,2017-11-11,Nate-Marquardt,109.250000,1108.0,1043.0,52.0,52.0,0.053192,0.000000,1.181680,0.651817,0.774472,0.228610,0.011025,0.013551,0.191621,0.182158,0.025957,0.045873,0.006714,0.714060,0.232936,0.234684,0.679408,0.260250,0.242022,12,11,0.285714,23,72.0,185.0,74.0,38.0,1,0,0
1313,Joe Soto vs Brett Johns,2017-12-01,Joe-Soto,29.216667,568.0,1115.0,17.0,20.0,0.000000,0.000000,1.294496,0.134138,2.245891,0.802553,0.005086,0.003537,0.145658,0.087121,0.062625,0.051232,0.000000,1.035366,0.089716,0.169415,1.156636,0.014669,0.123191,3,3,0.374532,6,66.0,135.0,65.0,30.0,1,0,0


In [66]:
fighter1 = fighter1.add_suffix('_1')
fighter2 = fighter2.add_suffix('_2')

In [67]:
fighter1.rename(columns={'date_1': 'date'}, inplace=True)
fighter2.rename(columns={'date_2': 'date'}, inplace=True)

fighter1.rename(columns={'fight_1': 'fight'}, inplace=True)
fighter2.rename(columns={'fight_2': 'fight'}, inplace=True)

In [68]:
fighter1

,fight,date,fighter_1,cumultime_1,cumulStr_Att_fighter_1,cumulStr_Att_oponent_1,cumulTD_Att_fighter_1,cumulTD_Att_oponent_1,KDPM_1,KDPM_ratio_1,SLPM_1,SLPM_ratio_1,SAPM_1,SAPM_ratio_1,Str_Acc_1,Str_Def_1,TD_Acc_1,TD_Def_1,SubPM_1,PassPM_1,RevPM_1,Head_SLPM_1,Body_SLPM_1,Leg_SLPM_1,Dist_SLPM_1,Clinch_SLPM_1,Ground_SLPM_1,wins_1,losses_1,implied_prob_1,prev_1,height_1,weight_1,reach_1,age_1,stance_Orthodox_1,stance_Southpaw_1,stance_Switch_1
1218,Erick Silva vs Neil Magny,2015-08-23,Neil-Magny,56.050000,980.0,528.0,44.0,54.0,0.005049,0.000000,1.760172,0.633346,0.704148,0.101334,0.004794,0.012334,0.120545,0.104416,0.005299,0.103628,0.009983,1.266204,0.288154,0.205814,0.861161,0.427905,0.471106,8,3,0.381679,11,75.0,170.0,80.0,28.0,1,0,0
512,Marc Diakiese vs Teemu Packalen,2017-03-18,Marc-Diakiese,12.666667,150.0,105.0,14.0,9.0,0.000000,0.000000,0.770234,0.215977,0.518134,0.209978,0.004523,0.011609,0.104396,0.148148,0.000000,0.033835,0.000000,0.557939,0.103494,0.108801,0.518797,0.192393,0.059045,2,0,0.666667,2,70.0,155.0,73.0,24.0,1,0,0
397,Tecia Torres vs Weili Zhang,2019-03-02,Weili-Zhang,10.683333,252.0,135.0,5.0,2.0,0.000000,0.000000,2.173216,0.680004,0.683362,0.060052,0.004654,0.007212,0.166667,1.000000,0.050826,0.064198,0.000000,1.329391,0.378469,0.465356,1.025648,0.129726,1.017842,2,0,0.555556,2,64.0,115.0,63.0,29.0,0,0,1
780,Urijah Faber vs Ricky Simon,2019-07-13,Urijah-Faber,98.066667,1607.0,1919.0,89.0,48.0,0.011927,0.000000,1.179982,0.352180,0.991396,0.173518,0.004591,0.005525,0.051524,0.091232,0.022279,0.031277,0.006395,0.881764,0.217457,0.080760,0.706166,0.149947,0.323869,10,6,0.238095,16,66.0,135.0,67.0,40.0,1,0,0
23,Anthony Pettis vs Jim Miller,2017-07-08,Anthony-Pettis,59.150000,867.0,945.0,15.0,65.0,0.019067,0.000000,1.241935,0.217224,1.172125,0.185897,0.005827,0.006997,0.044444,0.064790,0.016318,0.024383,0.028011,0.878335,0.227192,0.136408,1.058264,0.048451,0.135220,6,5,0.675676,11,70.0,170.0,72.0,30.0,1,0,0
386,Yair Rodriguez vs Alex Caceres,2016-08-06,Yair-Rodriguez,24.250000,340.0,261.0,18.0,20.0,0.012688,0.000000,1.153733,0.308279,0.675654,0.101955,0.005710,0.009397,0.141975,0.140476,0.041237,0.073411,0.011782,0.610853,0.283222,0.259658,0.827461,0.088365,0.237906,4,0,0.724638,4,71.0,145.0,71.0,23.0,1,0,0
668,Donald Cerrone vs Mike Perry,2018-11-10,Donald-Cerrone,134.150000,2471.0,2588.0,65.0,57.0,0.037986,0.022363,1.820667,0.244604,2.052632,0.310910,0.005634,0.005881,0.125775,0.245865,0.019975,0.041051,0.002613,0.895784,0.435111,0.489772,1.520768,0.145459,0.154440,20,8,0.363636,28,73.0,155.0,73.0,35.0,1,0,0
335,Michael Johnson vs Khabib Nurmagomedov,2016-11-12,Khabib-Nurmagomedov,38.683333,581.0,411.0,75.0,12.0,0.019742,0.000000,1.443705,0.832486,0.522134,0.090762,0.006003,0.012059,0.036431,0.319444,0.013640,0.144331,0.000000,1.247599,0.111088,0.085018,0.469735,0.090816,0.883154,7,0,0.735294,7,70.0,155.0,70.0,28.0,1,0,0
1082,Nate Marquardt vs Cezar Ferreira,2017-11-11,Cezar-Ferreira,54.550000,463.0,511.0,29.0,13.0,0.005238,0.000000,0.685717,0.190023,0.924717,0.290569,0.009524,0.012647,0.161494,0.365385,0.034399,0.044523,0.000000,0.446521,0.157264,0.081932,0.584622,0.019911,0.081184,7,4,0.729927,11,73.0,185.0,78.0,32.0,0,1,0
1313,Joe Soto vs Brett Johns,2017-12-01,Brett-Johns,14.000000,184.0,243.0,20.0,0.0,0.000000,0.000000,1.071429,0.208895,0.826531,0.112974,0.006255,0.005525,0.066667,NaN,0.010204,0.051020,0.000000,0.989796,0.051020,0.030612,0.469388,0.000000,0.602041,2,0,0.617284,2,67.0,135.0,71.0,25.0,1,0,0


In [69]:
fighter2

,fight,date,fighter_2,cumultime_2,cumulStr_Att_fighter_2,cumulStr_Att_oponent_2,cumulTD_Att_fighter_2,cumulTD_Att_oponent_2,KDPM_2,KDPM_ratio_2,SLPM_2,SLPM_ratio_2,SAPM_2,SAPM_ratio_2,Str_Acc_2,Str_Def_2,TD_Acc_2,TD_Def_2,SubPM_2,PassPM_2,RevPM_2,Head_SLPM_2,Body_SLPM_2,Leg_SLPM_2,Dist_SLPM_2,Clinch_SLPM_2,Ground_SLPM_2,wins_2,losses_2,implied_prob_2,prev_2,height_2,weight_2,reach_2,age_2,stance_Orthodox_2,stance_Southpaw_2,stance_Switch_2
1218,Erick Silva vs Neil Magny,2015-08-23,Erick-Silva,28.550000,233.0,377.0,8.0,33.0,0.060912,0.000000,2.311681,0.414093,1.815724,0.420818,0.025329,0.012281,0.208333,0.180014,0.099449,0.078695,0.000000,1.690404,0.564027,0.057249,0.665340,0.268043,1.378298,6,4,0.628931,10,72.0,170.0,74.0,31.0,1,0,0
512,Marc Diakiese vs Teemu Packalen,2017-03-18,Teemu-Packalen,7.400000,72.0,86.0,11.0,1.0,0.337838,0.000000,2.702703,0.109994,0.830116,0.166023,0.015545,0.017373,0.024793,1.000000,0.357143,0.511583,0.000000,2.567568,0.115830,0.019305,1.216216,0.000000,1.486486,1,1,0.343643,2,73.0,155.0,75.0,29.0,1,0,0
397,Tecia Torres vs Weili Zhang,2019-03-02,Tecia-Torres,57.883333,1143.0,1139.0,41.0,28.0,0.002468,0.000000,1.480200,0.266552,1.007694,0.121249,0.003925,0.005307,0.077236,0.148810,0.009173,0.033853,0.000000,0.884992,0.288431,0.306777,1.161782,0.230268,0.088150,6,3,0.416667,9,61.0,115.0,60.0,29.0,1,0,0
780,Urijah Faber vs Ricky Simon,2019-07-13,Ricky-Simon,21.000000,293.0,490.0,32.0,33.0,0.013605,0.000000,0.768707,0.109598,1.020408,0.206880,0.004082,0.004102,0.044866,0.052342,0.006803,0.054422,0.006803,0.489796,0.170068,0.108844,0.571429,0.081633,0.115646,3,0,0.714286,3,66.0,135.0,69.0,26.0,1,0,0
23,Anthony Pettis vs Jim Miller,2017-07-08,Jim-Miller,132.850000,1902.0,1869.0,74.0,61.0,0.008944,0.000000,1.018091,0.301817,0.897850,0.268107,0.004820,0.008376,0.137709,0.156740,0.059868,0.058093,0.005132,0.637816,0.192071,0.188204,0.674569,0.161144,0.182378,17,8,0.341297,25,68.0,155.0,71.0,33.0,0,1,0
386,Yair Rodriguez vs Alex Caceres,2016-08-06,Alex-Caceres,69.100000,1185.0,896.0,10.0,63.0,0.004244,0.000000,1.393131,0.384810,0.978804,0.163301,0.005183,0.008324,0.333333,0.089683,0.036436,0.037027,0.021068,0.863879,0.330884,0.198368,1.153009,0.153599,0.086523,7,6,0.289855,13,70.0,145.0,73.0,28.0,0,1,0
668,Donald Cerrone vs Mike Perry,2018-11-10,Mike-Perry,41.700000,783.0,943.0,8.0,18.0,0.064633,0.000000,1.696012,0.269137,1.611790,0.201156,0.004959,0.004952,0.239583,0.241667,0.000000,0.041212,0.000000,1.103540,0.220863,0.371609,1.248995,0.254775,0.192242,5,3,0.649351,8,70.0,170.0,71.0,27.0,1,0,0
335,Michael Johnson vs Khabib Nurmagomedov,2016-11-12,Michael-Johnson,80.466667,1854.0,1482.0,13.0,58.0,0.031919,0.012428,1.480057,0.308952,1.257626,0.253126,0.002845,0.006152,0.403846,0.181366,0.000000,0.000000,0.003551,0.829071,0.358896,0.292090,1.214812,0.144234,0.121012,9,6,0.271739,15,70.0,155.0,73.0,30.0,0,1,0
1082,Nate Marquardt vs Cezar Ferreira,2017-11-11,Nate-Marquardt,109.250000,1108.0,1043.0,52.0,52.0,0.053192,0.000000,1.181680,0.651817,0.774472,0.228610,0.011025,0.013551,0.191621,0.182158,0.025957,0.045873,0.006714,0.714060,0.232936,0.234684,0.679408,0.260250,0.242022,12,11,0.285714,23,72.0,185.0,74.0,38.0,1,0,0
1313,Joe Soto vs Brett Johns,2017-12-01,Joe-Soto,29.216667,568.0,1115.0,17.0,20.0,0.000000,0.000000,1.294496,0.134138,2.245891,0.802553,0.005086,0.003537,0.145658,0.087121,0.062625,0.051232,0.000000,1.035366,0.089716,0.169415,1.156636,0.014669,0.123191,3,3,0.374532,6,66.0,135.0,65.0,30.0,1,0,0


In [70]:

w1 = [1] * (len(fighter1)//2)
w2 = [2] * (len(fighter1) - len(w1))
w = w1 + w2

In [71]:

len(w)

1464

In [72]:
fighter1['winner'] = w
fighter2['winner'] = w

In [73]:
fighter1.to_csv('fighter1.csv')
fighter2.to_csv('fighter2.csv')

In [74]:
len(fighter1)

1464

In [75]:
len(fighter2)

1464

In [76]:
real_model_df = fighter1.merge(fighter2, how='outer', on=['fight', 'date', 'winner'], left_index=True, right_index=True)
real_model_df

,fight,date,fighter_1,cumultime_1,cumulStr_Att_fighter_1,cumulStr_Att_oponent_1,cumulTD_Att_fighter_1,cumulTD_Att_oponent_1,KDPM_1,KDPM_ratio_1,SLPM_1,SLPM_ratio_1,SAPM_1,SAPM_ratio_1,Str_Acc_1,Str_Def_1,TD_Acc_1,TD_Def_1,SubPM_1,PassPM_1,RevPM_1,Head_SLPM_1,Body_SLPM_1,Leg_SLPM_1,Dist_SLPM_1,Clinch_SLPM_1,Ground_SLPM_1,wins_1,losses_1,implied_prob_1,prev_1,height_1,weight_1,reach_1,age_1,stance_Orthodox_1,stance_Southpaw_1,stance_Switch_1,winner,fighter_2,cumultime_2,cumulStr_Att_fighter_2,cumulStr_Att_oponent_2,cumulTD_Att_fighter_2,cumulTD_Att_oponent_2,KDPM_2,KDPM_ratio_2,SLPM_2,SLPM_ratio_2,SAPM_2,SAPM_ratio_2,Str_Acc_2,Str_Def_2,TD_Acc_2,TD_Def_2,SubPM_2,PassPM_2,RevPM_2,Head_SLPM_2,Body_SLPM_2,Leg_SLPM_2,Dist_SLPM_2,Clinch_SLPM_2,Ground_SLPM_2,wins_2,losses_2,implied_prob_2,prev_2,height_2,weight_2,reach_2,age_2,stance_Orthodox_2,stance_Southpaw_2,stance_Switch_2
1218,Erick Silva vs Neil Magny,2015-08-23,Neil-Magny,56.050000,980.0,528.0,44.0,54.0,0.005049,0.000000,1.760172,0.633346,0.704148,0.101334,0.004794,0.012334,0.120545,0.104416,0.005299,0.103628,0.009983,1.266204,0.288154,0.205814,0.861161,0.427905,0.471106,8,3,0.381679,11,75.0,170.0,80.0,28.0,1,0,0,1,Erick-Silva,28.550000,233.0,377.0,8.0,33.0,0.060912,0.000000,2.311681,0.414093,1.815724,0.420818,0.025329,0.012281,0.208333,0.180014,0.099449,0.078695,0.000000,1.690404,0.564027,0.057249,0.665340,0.268043,1.378298,6,4,0.628931,10,72.0,170.0,74.0,31.0,1,0,0
512,Marc Diakiese vs Teemu Packalen,2017-03-18,Marc-Diakiese,12.666667,150.0,105.0,14.0,9.0,0.000000,0.000000,0.770234,0.215977,0.518134,0.209978,0.004523,0.011609,0.104396,0.148148,0.000000,0.033835,0.000000,0.557939,0.103494,0.108801,0.518797,0.192393,0.059045,2,0,0.666667,2,70.0,155.0,73.0,24.0,1,0,0,1,Teemu-Packalen,7.400000,72.0,86.0,11.0,1.0,0.337838,0.000000,2.702703,0.109994,0.830116,0.166023,0.015545,0.017373,0.024793,1.000000,0.357143,0.511583,0.000000,2.567568,0.115830,0.019305,1.216216,0.000000,1.486486,1,1,0.343643,2,73.0,155.0,75.0,29.0,1,0,0
397,Tecia Torres vs Weili Zhang,2019-03-02,Weili-Zhang,10.683333,252.0,135.0,5.0,2.0,0.000000,0.000000,2.173216,0.680004,0.683362,0.060052,0.004654,0.007212,0.166667,1.000000,0.050826,0.064198,0.000000,1.329391,0.378469,0.465356,1.025648,0.129726,1.017842,2,0,0.555556,2,64.0,115.0,63.0,29.0,0,0,1,1,Tecia-Torres,57.883333,1143.0,1139.0,41.0,28.0,0.002468,0.000000,1.480200,0.266552,1.007694,0.121249,0.003925,0.005307,0.077236,0.148810,0.009173,0.033853,0.000000,0.884992,0.288431,0.306777,1.161782,0.230268,0.088150,6,3,0.416667,9,61.0,115.0,60.0,29.0,1,0,0
780,Urijah Faber vs Ricky Simon,2019-07-13,Urijah-Faber,98.066667,1607.0,1919.0,89.0,48.0,0.011927,0.000000,1.179982,0.352180,0.991396,0.173518,0.004591,0.005525,0.051524,0.091232,0.022279,0.031277,0.006395,0.881764,0.217457,0.080760,0.706166,0.149947,0.323869,10,6,0.238095,16,66.0,135.0,67.0,40.0,1,0,0,1,Ricky-Simon,21.000000,293.0,490.0,32.0,33.0,0.013605,0.000000,0.768707,0.109598,1.020408,0.206880,0.004082,0.004102,0.044866,0.052342,0.006803,0.054422,0.006803,0.489796,0.170068,0.108844,0.571429,0.081633,0.115646,3,0,0.714286,3,66.0,135.0,69.0,26.0,1,0,0
23,Anthony Pettis vs Jim Miller,2017-07-08,Anthony-Pettis,59.150000,867.0,945.0,15.0,65.0,0.019067,0.000000,1.241935,0.217224,1.172125,0.185897,0.005827,0.006997,0.044444,0.064790,0.016318,0.024383,0.028011,0.878335,0.227192,0.136408,1.058264,0.048451,0.135220,6,5,0.675676,11,70.0,170.0,72.0,30.0,1,0,0,1,Jim-Miller,132.850000,1902.0,1869.0,74.0,61.0,0.008944,0.000000,1.018091,0.301817,0.897850,0.268107,0.004820,0.008376,0.137709,0.156740,0.059868,0.058093,0.005132,0.637816,0.192071,0.188204,0.674569,0.161144,0.182378,17,8,0.341297,25,68.0,155.0,71.0,33.0,0,1,0
386,Yair Rodriguez vs Alex Caceres,2016-08-06,Yair-Rodriguez,24.250000,340.0,261.0,18.0,20.0,0.012688,0.000000,1.153733,0.308279,0.675654,0.101955,0.005710,0.009397,0.141975,0.140476,0.041237,0.073411,0.011782,0.610853,0.283222,0.259658,0.827461,0.088365,0.237906,4,0,0.724638,4,71.0,145.0,71.0,23.0,1,0,0

In [77]:
real_model_df.to_csv('real_model_df.csv')

In [79]:
real_model_df.sort_values(by='date')

,fight,date,fighter_1,cumultime_1,cumulStr_Att_fighter_1,cumulStr_Att_oponent_1,cumulTD_Att_fighter_1,cumulTD_Att_oponent_1,KDPM_1,KDPM_ratio_1,SLPM_1,SLPM_ratio_1,SAPM_1,SAPM_ratio_1,Str_Acc_1,Str_Def_1,TD_Acc_1,TD_Def_1,SubPM_1,PassPM_1,RevPM_1,Head_SLPM_1,Body_SLPM_1,Leg_SLPM_1,Dist_SLPM_1,Clinch_SLPM_1,Ground_SLPM_1,wins_1,losses_1,implied_prob_1,prev_1,height_1,weight_1,reach_1,age_1,stance_Orthodox_1,stance_Southpaw_1,stance_Switch_1,winner,fighter_2,cumultime_2,cumulStr_Att_fighter_2,cumulStr_Att_oponent_2,cumulTD_Att_fighter_2,cumulTD_Att_oponent_2,KDPM_2,KDPM_ratio_2,SLPM_2,SLPM_ratio_2,SAPM_2,SAPM_ratio_2,Str_Acc_2,Str_Def_2,TD_Acc_2,TD_Def_2,SubPM_2,PassPM_2,RevPM_2,Head_SLPM_2,Body_SLPM_2,Leg_SLPM_2,Dist_SLPM_2,Clinch_SLPM_2,Ground_SLPM_2,wins_2,losses_2,implied_prob_2,prev_2,height_2,weight_2,reach_2,age_2,stance_Orthodox_2,stance_Southpaw_2,stance_Switch_2
1147,Ryan Bader vs Rogerio Nogueira,2010-09-25,Ryan-Bader,20.466667,314.0,253.0,20.0,0.0,0.046930,0.000000,1.189196,0.509609,0.498189,0.080115,0.005444,0.012539,0.089881,NaN,0.013960,0.100233,0.000000,0.830279,0.115079,0.243838,0.712250,0.159144,0.317802,4,0,0.769231,4,74.0,205.0,74.0,27.0,1,0,0,1,Rogerio-Nogueira,8.933333,95.0,64.0,1.0,13.0,0.057900,0.000000,1.178406,0.665423,0.461547,0.111593,0.008877,0.022277,1.000000,0.059172,0.015991,0.031983,0.047974,0.892765,0.063966,0.221675,1.098449,0.015991,0.063966,2,0,0.273973,2,74.0,205.0,75.0,34.0,0,1,0
939,Dan Henderson vs Mauricio Rua,2011-11-19,Dan-Henderson,33.200000,403.0,435.0,17.0,20.0,0.011254,0.000000,0.840690,0.151601,0.925876,0.182775,0.005828,0.005705,0.080882,0.110417,0.010040,0.028914,0.003347,0.506444,0.126940,0.207306,0.403615,0.243105,0.193971,3,2,0.392157,5,71.0,185.0,74.0,41.0,1,0,0,1,Mauricio-Rua,36.900000,452.0,487.0,20.0,17.0,0.036232,0.000000,1.161116,0.407508,1.097689,0.345949,0.008074,0.007932,0.045833,0.013072,0.008212,0.011669,0.000000,0.702672,0.169676,0.288768,0.654772,0.238595,0.267749,4,3,0.606061,7,73.0,205.0,76.0,29.0,1,0,0
940,Dan Henderson vs Mauricio Rua,2011-11-19,Mauricio-Rua,36.900000,452.0,487.0,20.0,17.0,0.036232,0.000000,1.161116,0.407508,1.097689,0.345949,0.008074,0.007932,0.045833,0.013072,0.008212,0.011669,0.000000,0.702672,0.169676,0.288768,0.654772,0.238595,0.267749,4,3,0.606061,7,73.0,205.0,76.0,29.0,1,0,0,2,Dan-Henderson,33.200000,403.0,435.0,17.0,20.0,0.011254,0.000000,0.840690,0.151601,0.925876,0.182775,0.005828,0.005705,0.080882,0.110417,0.010040,0.028914,0.003347,0.506444,0.126940,0.207306,0.403615,0.243105,0.193971,3,2,0.392157,5,71.0,185.0,74.0,41.0,1,0,0
1197,Renan Barao vs Urijah Faber,2012-07-21,Renan-Barao,18.150000,485.0,327.0,8.0,8.0,0.013276,0.000000,1.671859,0.369745,0.935596,0.088299,0.002573,0.006180,0.107143,0.250000,0.013276,0.029018,0.000000,1.077652,0.299569,0.294638,1.571528,0.023613,0.076718,3,0,0.769231,3,66.0,145.0,70.0,25.0,1,0,0,1,Urijah-Faber,18.450000,341.0,377.0,17.0,16.0,0.022123,0.000000,1.476934,0.421644,0.914772,0.129301,0.003900,0.006036,0.081283,0.084936,0.022123,0.044245,0.022123,1.151239,0.231551,0.094144,0.778718,0.152892,0.545324,2,1,0.246914,3,66.0,135.0,67.0,33.0,1,0,0
1198,Renan Barao vs Urijah Faber,2012-07-21,Urijah-Faber,18.450000,341.0,377.0,17.0,16.0,0.022123,0.000000,1.476934,0.421644,0.914772,0.129301,0.003900,0.006036,0.081283,0.084936,0.022123,0.044245,0.022123,1.151239,0.231551,0.094144,0.778718,0.152892,0.545324,2,1,0.246914,3,66.0,135.0,67.0,33.0,1,0,0,2,Renan-Barao,18.150000,485.0,327.0,8.0,8.0,0.013276,0.000000,1.671859,0.369745,0.935596,0.088299,0.002573,0.006180,0.107143,0.250000,0.013276,0.029018,0.000000,1.077652,0.299569,0.294638,1.571528,0.023613,0.076718,3,0,0.769231,3,66.0,135.0,70.0,25.0,1,0,0
342,Joe Lauzon vs Jim Miller,2012-12-29,Jim-Miller,66.083333,910.0,837.0,46.0,30.0,0.017980,0.000000,0.993483,0.320732,0.820672,0.317299,0.004771,0.009857,0.117184,0.167778,0.088393,0.080435,0.004324,0.651586,0.152012,0.189885,0.575181,0.152949,0.265353,10,3,0.444444,13,68.0,155.0,71.0,29.0,0,1,0,1,Joe-Lauzon,39.